[2025-01-17 Fiddler](https://thefiddler.substack.com/p/can-you-break-the-bell-curve)
====================

Fiddler
-------

After many rows, let $P_L$ be the probability of passing through the left slot
in a 3-pin row, and let $P_A$, $P_B$, and $P_C$ be the probabilities of passing
through each respective slot in a 2-pin row.

In [1]:
P_L, P_A, P_B, P_C = var('P_L, P_A, P_B, P_C')

The probabilities satisfy

In [2]:
eqns = [P_L == P_A + 1/4*P_B, P_A == 1/4*P_L, P_B == 3/4*P_L + 1/4*(1-P_L), P_C == 3/4*(1-P_L)]
show(eqns)

[P_L == P_A + 1/4*P_B,
 P_A == 1/4*P_L,
 P_B == 1/2*P_L + 1/4,
 P_C == -3/4*P_L + 3/4]

Solving,

In [3]:
solve(eqns, [P_L,P_A,P_B,P_C])

[[P_L == (1/10), P_A == (1/40), P_B == (3/10), P_C == (27/40)]]

The probability of landing in bucket A is 1/40.

Extra credit
------------

Since there are no solutions when the probability of going left when hitting a pin is 1/4 and
the probability of going right when hitting a pin is 3/4, I'll assume that the probabilities
are 1/2.

In [4]:
left = 1/2
right = 1/2

The probabilities of passing through each slot in the first 5-pin row from the top are

In [5]:
a,b,c = var('a,b,c')
p1 = (0,left*a,right*a+left*b,right*b+left*(1-a-b),right*(1-a-b),0)
show(p1)

(0, 1/2*a, 1/2*a + 1/2*b, -1/2*a + 1/2, -1/2*a - 1/2*b + 1/2, 0)

The probabilities of passing through each slot from the $i+1$th 5-pin row
from the top, given the probabilities of the passing through each slot
from the $i$th 5-pin row is given by

In [6]:
def next(p):
    q = (p[0] + left*p[1], right*p[1] + left*p[2], right*p[2] + left*p[3], right*p[3] + left*p[4], right*p[4] + p[5])
    return (left*q[0], right*q[0] + left*q[1], right*q[1] + left*q[2], right*q[2] + left*q[3], right*q[3] + left*q[4], right*q[4])

Looking for trapezoidal solutions in the first 100 5-pin rows,

In [7]:
p = p1
for i in [1..100]:
    for s in solve(p[1] - p[0] == p[2] - p[1], b):
        for s1 in solve((p[3] - p[4] == p[4] - p[5]).substitute(s), a):
            sa = s1.rhs()
            if sa >= 0 and sa <= 1:
                print (i,s1,s.substitute(s1),(c == 1-a-b).substitute(s).substitute(s1),[pp.substitute(s).substitute(s1) for pp in p])
    p = next(p)

1 a == (1/3) b == (1/3) c == (1/3) [0, 1/6, 1/3, 1/3, 1/6, 0]
3 a == (1/6) b == (2/3) c == (1/6) [3/64, 1/6, 55/192, 55/192, 1/6, 3/64]


(1/3,1/3,1/3) gives a trapezoidal distribution at the first 5-pin row from the top,
and (1/6,2/3,1/6) gives a trapezoidal distribution at the third 3-pin row from the top.

Another look
------------
Since it's been made clear that the probability of going left or right is 1/2, it's clear
that $a = c$, which means $b = 1 - 2a$.

And $x-y + x + x+y = 1/2$, so $x = 1/6$ for any row with a trapezoidal distribution.

In [8]:
pp1 = (0,1/2*a,1/2*a+1/2*(1-2*a))

In [9]:
def nextp(p):
    q = [p[0]+1/2*p[1],1/2*p[1]+1/2*p[2],p[2]]
    return [1/2*q[0],1/2*q[0]+1/2*q[1],1/2*q[1]+1/2*q[2]]    

In [10]:
pp = pp1
for i in [1..1000]:
    for s in solve(pp[1] - pp[0] == pp[2] - pp[1], a):
        aa = s.rhs()
        if aa >= 0 and aa <= 1/2:
            print(i,(aa,1-2*aa,aa),[p.substitute(a == aa) for p in pp])
    pp = nextp(pp)

1 (1/3, 1/3, 1/3) [0, 1/6, 1/3]
2 (1/3, 1/3, 1/3) [1/24, 1/6, 7/24]
3 (1/6, 2/3, 1/6) [3/64, 1/6, 55/192]


Looking at what the probability distribution will probably
converge to after many rows, solve for a fixed point,

In [11]:
p1,p2,p3 = var("p1,p2,p3")
p = nextp([p1,p2,p3])
solve([p1 == p[0], p2 == p[1], p3 == p[2], p1+p2+p3 == 1/2], [p1,p2,p3])

[[p1 == (1/10), p2 == (1/5), p3 == (1/5)]]

So the fixed point is (1/10,1/5,1/5,1/5,1/5,1/10).

Looking at the stability of the fixed point

In [12]:
nextp([1/10+p1,1/5+p2,1/5-p1-p2])

[1/2*p1 + 1/4*p2 + 1/10, 1/4*p1 + 1/4*p2 + 1/5, -3/4*p1 - 1/2*p2 + 1/5]

So it looks like the fixed point is stable, and all distributions eventually approach it,
so it looks like a trapezoidal distribution will not possible below some threshold row,
which I would guess is the third 5-pin row from the top.